In [2]:
!pip install google_patent_scraper

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.6/install/bin/python3.8 -m pip install --upgrade pip' command.


In [3]:
from google_patent_scraper import scraper_class
from bs4 import BeautifulSoup

In [4]:
import csv
import json

In [5]:
with open("2006_CN_KR_patent_grant_processed.csv", 'r',encoding="utf8") as file:
    csvreader = csv.reader(file)

    count = 0

    for row in csvreader:

        if count==0:
            print(len(row))

        count+=1

        if count<20:
            print(row)
        
print(count)

21
['', 'publication_id', 'publication_kind', 'publication_date', 'application_id', 'application_date', 'country', 'inventor_name', 'inventor_firstname', 'inventor_country', 'invention_title', 'number_of_claims', 'attorney_name', 'attorney_firstname', 'inventor males', 'inventor females', 'inventor team size', 'attorney males', 'attorney females', 'attorney team size', 'Year']
['2', 'US06982341', 'B1', '20060103', 'US11093995', '20050330', 'US', "['Kim Yunsoo', 'Kim Chang Gyoun', 'Chung Taek-Mo', 'Lee Sun Sook', 'An Ki-Seok', 'Yang Taek Seung', 'Jang Hong Suk']", "['Yunsoo', 'Chang Gyoun', 'Taek-Mo', 'Sun Sook', 'Ki-Seok', 'Taek Seung', 'Hong Suk']", "['KR', 'KR', 'KR', 'KR', 'KR', 'KR', 'KR']", 'Volatile copper aminoalkoxide complex and deposition of copper thin film using same', '6', '[]', '[]', '6', '1', '7', '0', '0', '0', '2006']
['10', 'US06982347', 'B2', '20060103', 'US10806460', '20040323', 'DE', "['Dieterle Martin', 'Petzoldt Jochen', 'Müller-Engel Klaus Joachim', 'Arnold Heik

In [6]:
with open("2020_CN_KR_patent_grant_processed.csv", 'r',encoding="utf8") as file:
    csvreader = csv.reader(file)

    count = 0

    for row in csvreader:

        if count==0:
            print(len(row))

        count+=1

        if count<20:
            print(row)
        
print(count)

21
['', 'publication_id', 'publication_kind', 'publication_date', 'application_id', 'application_date', 'country', 'inventor_name', 'inventor_firstname', 'inventor_country', 'invention_title', 'number_of_claims', 'attorney_name', 'attorney_firstname', 'inventor males', 'inventor females', 'inventor team size', 'attorney males', 'attorney females', 'attorney team size', 'Year']
['8', 'US10529746', 'B2', '20200107', 'US16106826', '20180821', 'KR', "['Bae Sung-hwan', 'Oh Seunghee', 'Kim Joonhyeong', 'Song Younggoo']", "['Sung-hwan', 'Seunghee', 'Joonhyeong', 'Younggoo']", "['KR', 'KR', 'KR', 'KR']", 'Display panel including external conductive pad, display apparatus including the same and method of manufacturing the same', '19', '[]', '[]', '1', '3', '4', '0', '0', '0', '2020']
['9', 'US10529747', 'B2', '20200107', 'US16184075', '20181108', 'KR', "['Han Se Hee', 'Kim Tae Gyun']", "['Se Hee', 'Tae Gyun']", "['KR', 'KR']", 'Display device', '9', '[]', '[]', '2', '0', '2', '0', '0', '0', '20

In [7]:
def return_cn_kr_numbers(number):
    
    if len(number)==0:
        return ["No Application Number"], ["No Application Number"]
    
    if number[0:4] == "USD0":
        number = number.replace('0', '', 1)
    elif number[0:3] == "US0":
        number = number.replace('0', '', 1)
    
    # ~ Initialize scraper class ~ #
    scraper=scraper_class()

    # ~~ Scrape patents individually ~~ #
    err, soup, url = scraper.request_single_patent(number)
    
    if err==404:
        return ["Patent not found"], ["Patent not found"]

    app_by_year = soup.find_all('li', itemprop='applicationsByYear')

    apps = []
    for x in app_by_year:
        ap = x.find_all('li', itemprop='application')
        apps = apps + ap

    patents = []
    #urls = []
    
    print(len(apps))

    for y in apps:
        if y.find_all('span',itemprop='countryCode')[0].text == 'CN' or y.find_all('span',itemprop='countryCode')[0].text == 'KR':
            url = y.find_all('span',itemprop='documentId')[0].text
            #urls.append(url)
            patents.append(url[7:])

    #print(urls)
    print(patents)
    #print("\n")
    
    if len(patents)==0:
        return ["No KR or CN Application Number"], ["No KR or CN Application Number"]
    
    names = []
    
    for p in patents:

        err_a, soup_a, url_a = scraper.request_single_patent(p)

        # ~ Parse results of scrape ~ #
        patent_p = scraper.get_scraped_data(soup_a,p,url_a)

        n = []
        
        # ~ Print inventors of patents ~ 
        for inventor in json.loads(patent_p['inventor_name']):
            #print('Patent inventor : {0}'.format(inventor['inventor_name']))
            n.append(inventor['inventor_name'])
        
        names.append(n)
    print(names)

    print("\n")
    return patents, names

In [ ]:
cn_kr_patents_rows = []

with open("2020_CN_KR_patent_grant_processed.csv", 'r',encoding="utf8") as file:
    csvreader = csv.reader(file)
    
    cn_kr_patents_rows = []
    
    count = 0

    for row in csvreader:
        
        if count%100==0 and count>0:
            with open("2020_CN_KR_patent_grant_processed_names.csv", mode='a', newline='',encoding="utf8") as file_3:
                writer = csv.writer(file_3)
                writer.writerows(cn_kr_patents_rows)
                print(count)
                print("="*50+"SAVED"+"="*50)
                cn_kr_patents_rows = []

        if count==0:
            
            row.append("WorldWide Applications")
            row.append("Original Names")
            
            cn_kr_patents_rows.append(row)
            
            print(len(row))
            print(row)
            print("\n\n")
            count+=1
            continue
        
        try:
            patents, names = return_cn_kr_numbers(row[1])
        except Exception as e:
            print(e)
            patents, names = [str(e)], [str(e)]
        
        row.append(patents)
        row.append(names)
        
        print(count,"\n\n")
        
        count+=1
        
        cn_kr_patents_rows.append(row)


        #if count<20:
        #    print(row)

if len(cn_kr_patents_rows)>0:
    with open("2020_CN_KR_patent_grant_processed_names.csv", mode='a', newline='',encoding="utf8") as file_3:
        writer = csv.writer(file_3)
        writer.writerows(cn_kr_patents_rows)
        print(count)
        print("="*50+"SAVED"+"="*50)
        cn_kr_patents_rows = []

In [ ]:
"""cn_kr_patents_rows = []

with open("2006_CN_KR_patent_grant_processed.csv", 'r',encoding="utf8") as file:
    csvreader = csv.reader(file)
    
    cn_kr_patents_rows = []
    
    count = 0

    for row in csvreader:
        
        if count%100==0 and count>0:
            with open("2006_CN_KR_patent_grant_processed_names.csv", mode='a', newline='',encoding="utf8") as file_3:
                writer = csv.writer(file_3)
                writer.writerows(cn_kr_patents_rows)
                print(count)
                print("="*50+"SAVED"+"="*50)
                cn_kr_patents_rows = []

        if count==0:
            
            row.append("WorldWide Applications")
            row.append("Original Names")
            
            cn_kr_patents_rows.append(row)
            
            print(len(row))
            print(row)
            print("\n\n")
            count+=1
            continue
        
        try:
            patents, names = return_cn_kr_numbers(row[1])
        except Exception as e:
            print(e)
            patents, names = [str(e)], [str(e)]
        
        row.append(patents)
        row.append(names)
        
        print(count,"\n\n")
        
        count+=1
        
        cn_kr_patents_rows.append(row)


        #if count<20:
        #    print(row)

if len(cn_kr_patents_rows)>0:
    with open("2006_CN_KR_patent_grant_processed_names.csv", mode='a', newline='',encoding="utf8") as file_3:
        writer = csv.writer(file_3)
        writer.writerows(cn_kr_patents_rows)
        print(count)
        print("="*50+"SAVED"+"="*50)
        cn_kr_patents_rows = []"""
print()

In [9]:
with open("2020_CN_KR_patent_grant_processed_names.csv", 'r',encoding="utf8") as file:
    csvreader = csv.reader(file)

    count = 0

    for row in csvreader:

        if count==0:
            print(len(row))

        count+=1

        if count<20:
            print(row)
        
print(count)

23
['', 'publication_id', 'publication_kind', 'publication_date', 'application_id', 'application_date', 'country', 'inventor_name', 'inventor_firstname', 'inventor_country', 'invention_title', 'number_of_claims', 'attorney_name', 'attorney_firstname', 'inventor males', 'inventor females', 'inventor team size', 'attorney males', 'attorney females', 'attorney team size', 'Year', 'WorldWide Applications', 'Original Names']
['8', 'US10529746', 'B2', '20200107', 'US16106826', '20180821', 'KR', "['Bae Sung-hwan', 'Oh Seunghee', 'Kim Joonhyeong', 'Song Younggoo']", "['Sung-hwan', 'Seunghee', 'Joonhyeong', 'Younggoo']", "['KR', 'KR', 'KR', 'KR']", 'Display panel including external conductive pad, display apparatus including the same and method of manufacturing the same', '19', '[]', '[]', '1', '3', '4', '0', '0', '0', '2020', "['KR20170139211A/en', 'CN107479229A/en']", "[['배성환', '오승희', '김준형', '송영구'], ['裵盛焕', '吴昇禧', '金俊亨', '宋荣九']]"]
['9', 'US10529747', 'B2', '20200107', 'US16184075', '20181108'